In [92]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
import h5py

import tensorflow as tf

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical

In [101]:
#Import dataset from pickle

mydata = pd.read_pickle("dataset.pkl")
print(mydata.shape)

(449963, 358)


In [114]:
datafile = 'clean_dat3.h5'

In [115]:
#Clean, process and save the dataset.

x_dataset_labels = ['SEX', '_AGEG5YR', 'HTM4', 'WTKG3', 'ADDEPEV2']
# y_dataset_labels = ['CVDINFR4', 'ADDEPEV2', 'CVDCRHD4', 'CVDSTRK3', 'TOLDHI2', 'DIABETE3', 'HAVARTH3', 'BPHIGH4', 'CHCKIDNY', 'CHCSCNCR', 'CHCCOPD1']
y_dataset_labels = ['MENTHLTH']
all_labels = x_dataset_labels + y_dataset_labels

df = pd.DataFrame(mydata, columns=all_labels)

# Converting all datapoints to integers
df = df.apply(pd.to_numeric)

# Removing empty columns
df = df.dropna()
# df = df.fillna(0)

# Loading in data
x_dataset = pd.DataFrame(df, columns=x_dataset_labels)
y_dataset = pd.DataFrame(df, columns=y_dataset_labels)

# Normalizing x values
x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_dataset = pd.DataFrame(x_scaled)


print(x_dataset.shape)
x_dataset = x_dataset.values
# x_dataset = x_dataset[:,np.newaxis]

h = h5py.File(datafile, 'w')

x = h.create_dataset("x", x_dataset.shape, data=x_dataset)
y = h.create_dataset("y", y_dataset.shape, data=y_dataset)

print(y_dataset)

h.close()

(415951, 6)
        MENTHLTH
0           88.0
1           88.0
2           88.0
3           88.0
4           88.0
5           88.0
6           88.0
8           25.0
9            1.0
10          88.0
11          88.0
12          88.0
13          88.0
14          88.0
15          88.0
16          88.0
17          77.0
18          30.0
19          88.0
20          88.0
21          88.0
22          15.0
23          88.0
24          88.0
25          88.0
26          88.0
27           1.0
28          88.0
29          88.0
30          88.0
...          ...
449933      88.0
449934      88.0
449935      88.0
449936      88.0
449937       7.0
449938      30.0
449939      88.0
449940      30.0
449941      88.0
449942      88.0
449943      88.0
449944       5.0
449945      88.0
449946      30.0
449947      30.0
449948      88.0
449949      88.0
449950      88.0
449951      88.0
449952      88.0
449953      88.0
449954      88.0
449955      88.0
449956      28.0
449957      88.0
449958      88.0
44

In [112]:
#See the shape of the saved data

h = h5py.File(datafile)

X = h['x']
Y = h['y']

print(X.shape)
print(Y.shape)

print(X[0])
print(Y[0])

h.close()

(415951, 6)
(415951, 1)
[0.125      0.76923077 0.51034483 0.20133164 0.125      0.8877551 ]
[88.]


In [113]:
#Train the data

h = h5py.File(datafile)

X = h['x']
Y = h['y']



test_x = X[:5000]
test_y = Y[:5000]

train_x = X[5000:]
train_y = Y[5000:]


length = train_x.shape[1]
model = models.Sequential()

# Input - Layer
model.add(layers.Dense(20, activation = "relu", input_shape=(length, )))

# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

# Output- Layer
model.add(layers.Dense(len(y_dataset_labels), activation = "sigmoid"))
model.summary()

# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

with tf.device('/gpu:0'):
    results = model.fit(
     train_x, train_y,
     epochs= 50,
     batch_size = 32,
     validation_data = (test_x, test_y),
     shuffle="batch"
    )

print("Test-Accuracy:", np.mean(results.history["val_acc"]))


h.close()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 20)                140       
_________________________________________________________________
dropout_41 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 50)                1050      
_________________________________________________________________
dropout_42 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_83 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_84 (Dense)             (None, 1)                 51        
Total params: 3,791
Trainable params: 3,791
Non-trainable params: 0
_________________________________________________________________
Train on

KeyboardInterrupt: 

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))